<h1>Normalise Plasmid Experiment</h1>
This notebook loads the data from the csv of plasmid concentrations so that it can be added to the protocol as a list the robot can read as we cannot put CSV files on the robot. It also checks how much buffer is used and how much is added to each well. 

In [3]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("PlateLayout362025.csv")

# Drop rows with missing or non-numeric concentrations
df = df[pd.to_numeric(df["conc"], errors="coerce").notnull()]

# Convert to dictionary
concentration_dict = dict(zip(df["well"], df["conc"].astype(float)))


# Preview it
print(concentration_dict)


{'A1': 63.45, 'A2': 59.1, 'A3': 46.8, 'A4': 69.0, 'A5': 30.0, 'A6': 28.7, 'A7': 61.0, 'A8': 84.4, 'A9': 32.0, 'A10': 35.0, 'A11': 69.0, 'A12': 32.0, 'B1': 33.0, 'B2': 108.0, 'B3': 65.0, 'B4': 34.0, 'B5': 29.0, 'B6': 66.0, 'B7': 43.0, 'B8': 44.0, 'B9': 50.5, 'B10': 44.0, 'B11': 70.0, 'B12': 64.0, 'C1': 41.0, 'C2': 26.0, 'C3': 20.0, 'C4': 73.0, 'C5': 22.0, 'C6': 78.0, 'C7': 25.0, 'C8': 12.5, 'C9': 53.0, 'C10': 50.95, 'C11': 35.0, 'C12': 60.0, 'D1': 24.0, 'D2': 50.0, 'D3': 132.0, 'D4': 17.0, 'D5': 62.0, 'D6': 19.0, 'D7': 23.0, 'D8': 65.0, 'D9': 62.0, 'D10': 48.0, 'D11': 49.0, 'D12': 39.0, 'E1': 54.0, 'E2': 29.95, 'E3': 55.0, 'E4': 32.0, 'E5': 39.0, 'E6': 45.0, 'E7': 75.0, 'E8': 54.0, 'E9': 15.0, 'E10': 14.0, 'E11': 44.0, 'E12': 19.0, 'F1': 36.0, 'F2': 38.0, 'F3': 160.0, 'F4': 33.0}


Function to calculate buffer usage and overflows

In [2]:
def simulate_buffer_usage(csv_path, initial_volume, target_concentration):
    concentrations = {}
    with open(csv_path, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            well = row["well"]
            conc = float(row["conc"])
            concentrations[well] = conc

    total_buffer_used = 0.0
    for well_name, c1 in concentrations.items():
        if c1 <= target_concentration:
            continue
        v2 = (c1 * initial_volume) / target_concentration
        if v2 > 300:
            print(f"[WARNING] {well_name}: final volume {round(v2, 2)} µL exceeds 300 µL — skipping.")
            continue
        buffer_volume = v2 - initial_volume
        total_buffer_used += buffer_volume
        print(f"[SIMULATION] {well_name}: buffer used = {round(buffer_volume, 2)} µL")

    print(f"[SIMULATION] Total buffer required: {round(total_buffer_used, 2)} µL")

buffer_used_tracker = {"total": 0.0}

In [3]:
from opentrons import protocol_api
import csv
metadata = {
    "protocolName": "Plasmid concentration Normalisation",
    "description": "A protocol to normalise plasmid concentrations in a 96 well plate using Opentrons Flex, should only use one tip. Requires a list of plasmid concentrations and locations. Run the associated notebook first to calculate the amount of buffer you need and to set the intial volumes low enough to prevent overflow in the 96 well plate",
    "author": "JATD"
}

requirements = {
    "robotType": "Flex", # Ensure the robot type is Flex,
    "apiLevel": "2.23",
    }


def run(protocol: protocol_api.ProtocolContext):
    concentrations = {'A1': 63.45, 'A2': 59.1, 'A3': 46.8, 'A4': 69.0, 'A5': 30.0, 'A6': 28.7, 'A7': 61.0, 'A8': 84.4, 'A9': 32.0, 'A10': 35.0, 'A11': 69.0, 'A12': 32.0, 'B1': 33.0, 'B2': 108.0, 'B3': 65.0, 'B4': 34.0, 'B5': 29.0, 'B6': 66.0, 'B7': 43.0, 'B8': 44.0, 'B9': 50.5, 'B10': 44.0, 'B11': 70.0, 'B12': 64.0, 'C1': 41.0, 'C2': 26.0, 'C3': 20.0, 'C4': 73.0, 'C5': 22.0, 'C6': 78.0, 'C7': 25.0, 'C8': 12.5, 'C9': 53.0, 'C10': 50.95, 'C11': 35.0, 'C12': 60.0, 'D1': 24.0, 'D2': 50.0, 'D3': 132.0, 'D4': 17.0, 'D5': 62.0, 'D6': 19.0, 'D7': 23.0, 'D8': 65.0, 'D9': 62.0, 'D10': 48.0, 'D11': 49.0, 'D12': 39.0, 'E1': 54.0, 'E2': 29.95, 'E3': 55.0, 'E4': 32.0, 'E5': 39.0, 'E6': 45.0, 'E7': 75.0, 'E8': 54.0, 'E9': 15.0, 'E10': 14.0, 'E11': 44.0, 'E12': 19.0, 'F1': 36.0, 'F2': 38.0, 'F3': 160.0, 'F4': 33.0}

    tips = protocol.load_labware("opentrons_flex_96_tiprack_50ul", "C2")
    tubes = protocol.load_labware("opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap", "D2")
    trash = protocol.load_trash_bin("A3")
    
    temp_module = protocol.load_module("temperature module gen2", "D1")
    plate1 = temp_module.load_labware("corning_96_wellplate_360ul_flat")
    #temp_module.set_temperature(24)

    left_pipette = protocol.load_instrument(
        "flex_1channel_50",
        mount="left",
        tip_racks=[tips]
    )

    initial_volume = 25  # µL
    target_concentration = 20  # ng/µL
    buffer_volume_per_tube = 1000  # µL per tube initially
    tube_index = 0
    buffer_tube = tubes.wells()[tube_index]
    buffer_left_in_tube = buffer_volume_per_tube
    left_pipette.pick_up_tip()

    for well_name, c1 in concentrations.items():
        if c1 <= target_concentration:
            continue

        v2 = (c1 * initial_volume) / target_concentration
        buffer_volume = v2 - initial_volume

        if buffer_volume > 0:
            # Check if buffer left is enough, else switch tube
            if buffer_volume > buffer_left_in_tube or buffer_left_in_tube < 50:
                tube_index += 1
                if tube_index >= len(tubes.wells()):
                    raise RuntimeError("Not enough buffer in tubes to complete the protocol!")
                buffer_tube = tubes.wells()[tube_index]
                buffer_left_in_tube = buffer_volume_per_tube

            #left_pipette.pick_up_tip()
            left_pipette.transfer(buffer_volume, 
                                  buffer_tube, 
                                  plate1.wells_by_name()[well_name].top(-1),
                                    new_tip='never')
            #left_pipette.drop_tip()

            buffer_left_in_tube -= buffer_volume

if __name__ == "__main__":
    simulate_buffer_usage("PlateLayout362025.csv", initial_volume=25, target_concentration=20)

[SIMULATION] A1: buffer used = 54.31 µL
[SIMULATION] A2: buffer used = 48.88 µL
[SIMULATION] A3: buffer used = 33.5 µL
[SIMULATION] A4: buffer used = 61.25 µL
[SIMULATION] A5: buffer used = 12.5 µL
[SIMULATION] A6: buffer used = 10.88 µL
[SIMULATION] A7: buffer used = 51.25 µL
[SIMULATION] A8: buffer used = 80.5 µL
[SIMULATION] A9: buffer used = 15.0 µL
[SIMULATION] A10: buffer used = 18.75 µL
[SIMULATION] A11: buffer used = 61.25 µL
[SIMULATION] A12: buffer used = 15.0 µL
[SIMULATION] B1: buffer used = 16.25 µL
[SIMULATION] B2: buffer used = 110.0 µL
[SIMULATION] B3: buffer used = 56.25 µL
[SIMULATION] B4: buffer used = 17.5 µL
[SIMULATION] B5: buffer used = 11.25 µL
[SIMULATION] B6: buffer used = 57.5 µL
[SIMULATION] B7: buffer used = 28.75 µL
[SIMULATION] B8: buffer used = 30.0 µL
[SIMULATION] B9: buffer used = 38.12 µL
[SIMULATION] B10: buffer used = 30.0 µL
[SIMULATION] B11: buffer used = 62.5 µL
[SIMULATION] B12: buffer used = 55.0 µL
[SIMULATION] C1: buffer used = 26.25 µL
[SIMU

Simulate the protocol using the opentrons simulator function to test if it will run on the robot

In [4]:
!opentrons_simulate normalise_plasmid_concentrations.py

/Users/jackdalton/.opentrons/robot_settings.json not found. Loading defaults
Belt calibration not found.
Picking up tip from A1 of Opentrons Flex 96 Tip Rack 50 µL on slot C2
Transferring 54.3125 from A1 of Opentrons 24 Tube Rack with Eppendorf 1.5 mL Safe-Lock Snapcap on slot D2 to A1 of Corning 96 Well Plate 360 µL Flat on Temperature Module GEN2 on slot D1
	Aspirating 27.15625 uL from A1 of Opentrons 24 Tube Rack with Eppendorf 1.5 mL Safe-Lock Snapcap on slot D2 at 35.0 uL/sec
	Dispensing 27.15625 uL into A1 of Corning 96 Well Plate 360 µL Flat on Temperature Module GEN2 on slot D1 at 57.0 uL/sec
	Aspirating 27.15625 uL from A1 of Opentrons 24 Tube Rack with Eppendorf 1.5 mL Safe-Lock Snapcap on slot D2 at 35.0 uL/sec
	Dispensing 27.15625 uL into A1 of Corning 96 Well Plate 360 µL Flat on Temperature Module GEN2 on slot D1 at 57.0 uL/sec
Transferring 48.875 from A1 of Opentrons 24 Tube Rack with Eppendorf 1.5 mL Safe-Lock Snapcap on slot D2 to A2 of Corning 96 Well Plate 360 µL Fla